In [1]:
!python starter.py

In [2]:
from starter import *
library_details()

OpenCV:  4.5.3
Numpy:  1.19.5


In [63]:
labels = ['arduino', 'microbit', 'esp32']
# labels = ['arduino']
number_imgs = 150
print(labels)

['arduino', 'microbit', 'esp32']


In [15]:
IMAGES_PATH = os.path.join('..', 'ssd_workspace', 'image_dataset', 'images')

..\ssd_workspace\image_dataset\images


In [16]:
if not os.path.exists(IMAGES_PATH):
    !mkdir {IMAGES_PATH}

In [17]:
VIDEOS_PATH = []
VIDEOS_PATH.append(os.path.join('..', 'ssd_workspace','videos', 'arduino.mp4'))
VIDEOS_PATH.append(os.path.join('..', 'ssd_workspace','videos','microbit.mp4'))
VIDEOS_PATH.append(os.path.join('..', 'ssd_workspace','videos','esp32.mp4'))
print(VIDEOS_PATH)

['..\\ssd_workspace\\videos\\arduino.mp4', '..\\ssd_workspace\\videos\\microbit.mp4', '..\\ssd_workspace\\videos\\esp32.mp4']


In [12]:
videos = {}
for key in labels:
    for value in VIDEOS_PATH:
        videos[key] = value
        VIDEOS_PATH.remove(value)
        break
print(videos)

{'arduino': '..\\ssd_workspace\\videos\\arduino.mp4', 'microbit': '..\\ssd_workspace\\videos\\microbit.mp4', 'esp32': '..\\ssd_workspace\\videos\\esp32.mp4'}


In [64]:
for label in labels: 
    cap = cv2.VideoCapture(videos[label])
    print("Collecting images for {}".format(label))
    time.sleep(5)
    num = 0
    counter = 0
    for img_num in range(number_imgs):
        if (img_num%10 == 0):
            #print("collecting image {}".format(num))
            ret, frame = cap.read()
            img_name = os.path.join(IMAGES_PATH, label+'_'+'{}.jpg'.format(num))
            cv2.imwrite(img_name, frame)
            cv2.imshow('frame', frame)
            num = num + 1

        counter = counter + 1
    cap.release()
    cv2.destroyAllWindows()

In [20]:
cv2.destroyAllWindows()

In [21]:
img_name

'..\\ssd_workspace\\image_dataset\\images\\arduino\\arduino_16.jpg'

In [22]:
IMAGES_PATH

'..\\ssd_workspace\\image_dataset\\images'

In [37]:
for label in labels:
    cap = cv2.VideoCapture(videos[label])
    numb = 0
    while(True):
        ret, frame = cap.read()
        cv2.imshow('frame', frame)
        numb = numb + 1
cap.release()
cv2.destroyAllWindows()        
print(numb)

error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-c2l3r8zm\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [62]:
cap.release()
cv2.destroyAllWindows()

In [50]:
# importing libraries
import cv2
import numpy as np

# Create a VideoCapture object and read from input file
cap = cv2.VideoCapture(videos['arduino'])
numb = 0

# Check if camera opened successfully
if (cap.isOpened()== False):
    print("Error opening video file")

# Read until video is completed
while(cap.isOpened()):
    numb = numb + 1
	
# Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        

	# Display the resulting frame
        cv2.imshow('Frame', frame)
        

	# Press Q on keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

# Break the loop
    else:
        break

# When everything done, release
# the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()
print(numb)

68


In [39]:
videos['arduino']

'..\\ssd_workspace\\videos\\arduino.mp4'